In [1]:
import mayavi
import numpy as np
import scipy
from numpy import linalg as la
from scipy import linalg
from mayavi import mlab

ModuleNotFoundError: No module named 'mayavi'

In [13]:
''' Joint Element Density of 2x2 B = exp(A) where A = [[a, c], [c, b]] and is sampled from the GOE(2)'''
''' Obtained via a change of variables for the inverse trnasformation (B -> logB = A) on the known density of A ~ GOE '''

'''Jacobian of inverse transformation can be found in Schwartzman: 
Lognormal Distributions and Geometric Averages of Symmetric Positive Definite Matrices'''

def g(lambdai, lambdaj):
    
    '''Function of the spectrum of B = exp(A): lambda_B = exp(lambda_A)'''
    
    if lambdai == lambdaj:
        return 1 / lambdai
    elif lambdai != lambdaj:
        if lambdai > lambdaj:
            return (np.log(lambdai) - np.log(lambdaj)) / (lambdai - lambdaj)
        elif lambdaj > lambdai:
            return (np.log(lambdaj) - np.log(lambdai)) / (lambdaj - lambdai)
    
def f(a,b,c):
    
    A = np.asarray([[a, c], 
                    [c, b]])
    
    B = scipy.linalg.expm(A)
    
    eigs = la.eigvalsh(B)
    
    #reciprocal of determinant of B
    J_1 = 1 / la.det(B)
    #function of spectrum
    J_2 = g(eigs[0], eigs[1])
    #together these terms make up the Jacobian of the transformation
    J = J_1 * J_2
    
    frb_nrm = la.norm(scipy.linalg.logm(B), ord = 'fro')
    
    
    return J * 1/(np.sqrt(4 * np.pi**3)) * np.exp(-frb_nrm/2)
    

In [14]:
'''Given the density, we wish to construct a 3D density plot to visualize the transformation of the exponential mapping'''



nrepeat = 20000 
mu, sigma = 0, 1
density = np.zeros(nrepeat)
a = np.random.normal(mu, sigma, nrepeat)
b = np.random.normal(mu, sigma, nrepeat)
c = np.random.normal(mu, sigma/2, nrepeat)

x = np.zeros(nrepeat)
y = np.zeros(nrepeat)
z = np.zeros(nrepeat)


for i in range(nrepeat):
    
    density[i] = (f(a[i],b[i],c[i]))
    
    A = np.asarray([[a[i], c[i]], [c[i], b[i]]])
    B = scipy.linalg.expm(A)
    
    x[i] = B[0,0]
    y[i] = B[1,1]
    z[i] = B[0,1]

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, density, scale_mode='none', scale_factor=0.07) #
mask = pts.glyph.mask_points
mask.maximum_number_of_points = x.size
mask.on_ratio = 1
pts.glyph.mask_input_points = True

figure.scene.disable_render = False 
mlab.axes()
mlab.show()


libpng warning: iCCP: known incorrect sRGB profile


In [11]:
'''Compare to GOE density function'''

def h(a,b,c):
    
    frb_nrm = (abs(a)**2 + abs(b)**2 + 2*abs(c)**2)
    
    return 1/(np.sqrt(4 * np.pi**3)) * np.exp(-(frb_nrm)/2)

density_2 = np.zeros(nrepeat)

for i in range(nrepeat):
    density_2[i] = (h(a[i], b[i], c[i]))

figure = mlab.figure('DensityPlot')
pts = mlab.points3d(a, b, c, density, scale_mode='none', scale_factor=0.07)
mlab.axes()
mlab.show()

NameError: name 'xmin' is not defined